In [6]:
import os
from PIL import Image

from torchvision import transforms

In [13]:
# Scale range of the cropped image before resizing,
# relatively to the origin image. Used for large global view cropping
global_crops_scale=(0.4, 1.)
# Number of small local views to generate
local_crops_number=8  
# Scale range of the cropped image before resizing,
# relatively to the origin image. Used for small local view cropping of multi-crop.
local_crops_scale=(0.05, 0.4)

class DataAugmentationDINO(object):
    """ Data augmentation strategy used in DINO
    Some augmentation were removed for simplicity
    """
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [
                    transforms.ColorJitter(
                        brightness=0.4,
                        contrast=0.4,
                        saturation=0.2,
                        hue=0.1
                    )
                ],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])
        normalize = transforms.Compose([
            transforms.ToTensor(),
            #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # first global crop
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(
                224, scale=global_crops_scale, interpolation=Image.BICUBIC
            ),
            flip_and_color_jitter,
            #utils.GaussianBlur(1.0),
            normalize,
        ])
        # second global crop
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(
                224, scale=global_crops_scale, interpolation=Image.BICUBIC
            ),
            flip_and_color_jitter,
            #utils.GaussianBlur(0.1),
            #utils.Solarization(0.2),
            normalize,
        ])
        # transformation for the local small crops
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(
                96, scale=local_crops_scale, interpolation=Image.BICUBIC
            ),
            flip_and_color_jitter,
            #utils.GaussianBlur(p=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops

In [14]:
image_path = "cat.jpg"
with open(image_path, 'rb') as f:
    img = Image.open(f)
    img = img.convert('RGB')

In [15]:
# Apply data augmentation
data_augmentation = DataAugmentationDINO(global_crops_scale, local_crops_scale, local_crops_number)
crops = data_augmentation(img)

In [16]:
SAVE_DIR = "crops"
os.makedirs(SAVE_DIR, exist_ok=True)
for idx, crop in enumerate(crops):
    # convert to PIL image
    crop = transforms.ToPILImage()(crop)
    crop.save(os.path.join(SAVE_DIR, f"crop_{idx}.jpg"))